In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.metrics import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import tensorflow as tf
import keras.utils as utils
import scipy.io as scipy

Using TensorFlow backend.


In [2]:
train_0_files = os.listdir('./train/0')
train_1_files = os.listdir('./train/1')

In [3]:
len(train_0_files)

12750

In [4]:
X_train = []
y_train = []

In [5]:
for file in train_0_files:
    path = os.path.join('./train/0',file)
    mat = scipy.loadmat(path)
    X_train.append(mat['eegdata'])
    y_train.append(0)

In [6]:
for file in train_1_files:
    path = os.path.join('./train/1',file)
    mat = scipy.loadmat(path)
    X_train.append(mat['eegdata'])
    y_train.append(1)

In [7]:
X = np.array(X_train)
y = np.array(y_train)

In [8]:
X.shape

(22950, 64, 160)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
X_train.shape

(18360, 64, 160)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [12]:
X_train.shape

(14688, 64, 160)

In [13]:
class DataLoader(utils.Sequence):
    def __init__(self, X, Y, batch_size = 64):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        
    def on_epoch_end(self):
        perm_inds = np.arange(len(self.X))
        np.random.shuffle(perm_inds)
        self.X = self.X[perm_inds]
        self.Y = self.Y[perm_inds]
    
    def process(self,y,arr):
        #print(y.shape)
        for i in range(64):
            y[i][0][4]=arr[i][21]
            y[i][0][5]=arr[i][22]
            y[i][0][6]=arr[i][23]
            y[i][1][3]=arr[i][24]
            y[i][1][4]=arr[i][25]
            y[i][1][5]=arr[i][26]
            y[i][1][6]=arr[i][27]
            y[i][1][7]=arr[i][28]
            y[i][2][1]=arr[i][29]
            y[i][2][2]=arr[i][30]
            y[i][2][3]=arr[i][31]
            y[i][2][4]=arr[i][32]
            y[i][2][5]=arr[i][33]
            y[i][2][6]=arr[i][34]
            y[i][2][7]=arr[i][35]
            y[i][2][8]=arr[i][36]
            y[i][2][9]=arr[i][37]
            y[i][3][1]=arr[i][38]
            y[i][3][2]=arr[i][0]
            y[i][3][3]=arr[i][1]
            y[i][3][4]=arr[i][2]
            y[i][3][5]=arr[i][3]
            y[i][3][6]=arr[i][4]
            y[i][3][7]=arr[i][5]
            y[i][3][8]=arr[i][6]
            y[i][3][9]=arr[i][39]
            y[i][4][0]=arr[i][42]
            y[i][4][1]=arr[i][40]
            y[i][4][2]=arr[i][7]
            y[i][4][3]=arr[i][8]
            y[i][4][4]=arr[i][9]
            y[i][4][5]=arr[i][10]
            y[i][4][6]=arr[i][11]
            y[i][4][7]=arr[i][12]
            y[i][4][8]=arr[i][13]
            y[i][4][9]=arr[i][41]
            y[i][4][10]=arr[i][43]
            y[i][5][1]=arr[i][44]
            y[i][5][2]=arr[i][14]
            y[i][5][3]=arr[i][15]
            y[i][5][4]=arr[i][16]
            y[i][5][5]=arr[i][17]
            y[i][5][6]=arr[i][18]
            y[i][5][7]=arr[i][19]
            y[i][5][8]=arr[i][20]
            y[i][5][9]=arr[i][45]
            y[i][6][1]=arr[i][46]
            y[i][6][2]=arr[i][47]
            y[i][6][3]=arr[i][48]
            y[i][6][4]=arr[i][49]
            y[i][6][5]=arr[i][50]
            y[i][6][6]=arr[i][51]
            y[i][6][7]=arr[i][52]
            y[i][6][8]=arr[i][53]
            y[i][6][9]=arr[i][54]
            y[i][7][3]=arr[i][55]
            y[i][7][4]=arr[i][56]
            y[i][7][5]=arr[i][57]
            y[i][7][6]=arr[i][58]
            y[i][7][7]=arr[i][59]
            y[i][8][4]=arr[i][60]
            y[i][8][5]=arr[i][61]
            y[i][8][6]=arr[i][62]
            y[i][9][5]=arr[i][63]
        return y
    
    def __len__(self):
        return len(self.X)//self.batch_size
    
    def __getitem__(self, index):
        X_batch = self.X[index*self.batch_size:(index+1)*self.batch_size]
        Y_batch = self.Y[index*self.batch_size:(index+1)*self.batch_size]
        y = np.zeros((64,10,11,160))
        #print(X_batch.shape)
        X_batch = self.process(y,X_batch)
        X_batch=X_batch.reshape(-1,160,10,11,1)
        #print(X_batch.shape)
        #X_batch=np.moveaxis(y,3,1)
        return X_batch, Y_batch

In [14]:
s = np.zeros((64,160,10,11))
s=s[:,:,:,:,None]
#s=np.moveaxis(s,3,1)

In [15]:
s.shape

(64, 160, 10, 11, 1)

In [16]:
#ConvLSTM Model
batch_size = 64
model = Sequential()

#model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')
model.add(BatchNormalization(input_shape=s.shape[1:]))
model.add(ConvLSTM2D(filters=3,kernel_size=(3,3),dropout=0.2,recurrent_dropout=0.1,return_sequences=True))
model.add(Activation('tanh'))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=3,kernel_size=(3,3),dropout=0.2,recurrent_dropout=0.1,return_sequences=True))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(rate=0.5))
model.add(Dense(units=1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 160, 10, 11, 1)    4         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 160, 8, 9, 3)      444       
_________________________________________________________________
activation_1 (Activation)    (None, 160, 8, 9, 3)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 160, 8, 9, 3)      12        
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 160, 6, 7, 3)      660       
_________________________________________________________________
batch_normalization_3 (Batch (None, 160, 6, 7, 3)      12        
_________________________________________________________________
flatten_1 (Flatten)          (None, 20160)             0         
__________

In [17]:
train_loader = DataLoader(X_train, y_train)
val_loader = DataLoader(X_val, y_val)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')
model.fit_generator(callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto'), ModelCheckpoint('./checkpoint/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=10)], validation_data=val_loader, generator = train_loader,use_multiprocessing=True,workers=6, verbose=1, shuffle = True, epochs=500)

Epoch 1/500
229/229 [==============================] - 261s 1s/step - loss: 1.1082 - val_loss: 0.8132
Epoch 2/500
229/229 [==============================] - 258s 1s/step - loss: 0.9709 - val_loss: 0.7270
Epoch 3/500
229/229 [==============================] - 259s 1s/step - loss: 0.8406 - val_loss: 0.7145
Epoch 4/500
229/229 [==============================] - 259s 1s/step - loss: 0.7581 - val_loss: 0.6877
Epoch 5/500
229/229 [==============================] - 259s 1s/step - loss: 0.7128 - val_loss: 0.6849
Epoch 6/500
229/229 [==============================] - 259s 1s/step - loss: 0.6878 - val_loss: 0.6792
Epoch 7/500
229/229 [==============================] - 258s 1s/step - loss: 0.6834 - val_loss: 0.6806
Epoch 8/500
229/229 [==============================] - 259s 1s/step - loss: 0.6816 - val_loss: 0.6813
Epoch 9/500
229/229 [==============================] - 259s 1s/step - loss: 0.6795 - val_loss: 0.6799
Epoch 10/500
229/229 [==============================] - 260s 1s/step - loss: 0.677

In [18]:
test_x = DataLoader(X_test, y_test)
#model.predict(test_x, batch_size=64, verbose=1)
y = np.zeros((len(X_test),10,11,160))

y = test_x.process(y,X_test)
y=y.reshape(-1,160,10,11,1)
y.shape

(4590, 160, 10, 11, 1)

In [19]:
y_pred = model.predict(y, verbose=1)
#for i in range(len(y_pred)):
#    if y_pred[i]>=0.5:
#        y_pred[i]=1
#    else:
#        y_pred[i]=0
y_pred = y_pred.round()
y_pred = y_pred.reshape(-1)
y_pred = y_pred.astype(int)

#binary_accuracy(y_test, y_pred)

4590/4590 [==============================] - 65s 14ms/step


In [20]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.55      0.99      0.71      2537
          1       0.52      0.01      0.01      2053

avg / total       0.54      0.55      0.40      4590

